# FPL Project

## Introduction to the English Premier League

The English Premier League (EPL) is ...

## Introduction to Fantasy Premier League

Fantasy Premier League (FPL) is an online strategy game that casts the player in the role of a Fantasy manager. Each manager assembles a virtual team of real-life football players in the English Premier League and score points based on their real-life performance in their Premier League matches. The goal for each manager in FPL is to maximise their points by the end of the season.

At the beginning of the season, managers are allocated a budget of £100 million to build their team of 15 players. Prices of players are assigned at the beginning of the season, then they fluctuate throughout the season based on their performance.

Points are awarded to players for goals, assists, saves, clean sheets as well as bonus points, which are awarded to the top-performing individuals in a match. Points can also be taken away for yellow cards, red cards and goals conceded. A team's points for the match round or "gameweek" will be scored by the starting XI. There are 38 gameweeks every season; one for each match.

## Project Goal & Constraints

FPL is a popular game amongst fans of the English Premier League, as Fantasy managers attempt to build the optimal combination of players every gameweek. Fantasy managers can only guess the optimal combination of players, because they must pick their teams before any real-life Premier League games occur. In other words, there is *uncertainty* regarding which players will perform the best. For this project, we will be removing this uncertainty by using the FPL dataset of the 2016-2017 season, which has already recorded the highest performing players for every gameweek.

This project is considered to be an optimisation problem, because we are attempting to maximise the total number of points of our team in the 2016-2017 season, whilst adhering to a multitude of different constraints.  There are many constraints that we must consider, which are:

1. Position Constraint: Each manager must select a team of 15 players, consisting of 2 goalkeepers, 5 defenders, 5 midfielders, and 3 forwards.

2. Price Constraint: The total price of the 15 players cannot exceed £100 million at the beginning of the season.

3. Formation Constraint: The starting XI of each team must play with a formation of 1 goalkeeper, 3-5 defeneders, 2-5 midfielders, and 1-3 forwards. The remaining four players would be on the bench, but the chosen formation can change before each gameweek.

4. Team Constraint: Each manager can have a maximum of three players from the same Premier League team at any point in the season.

5. Transfer Constraint: Each manager is allowed one free transfer per week. Any subsequent transfers results in a four-point deduction for that gameweek. A manager can transfer or "exchange" any of their players for a different player, given that the price constraint still holds.

These constraints will make the calculation of the optimal team more difficult, as we cannot just pick the highest-scroing players from each gameweek. We can classify these constraints into "team-building constraints" and the "team-evaluating constraint". The first four of these constraints interfere with the building of the team, in the sense that the team would be invalid of they are not satisfied. The last constraint, the transfer constraint is considered a "team-evaluating constraint" because it does not interfere with the building of the team. Having more transfers does not invalidate a team, but rather "punishes" teams for transfering more players by decreasing the total number of points

## Extra Rules

There are some extra rules in FPL that can help managers increase their points total. Captaincy should be assigned every week and the Wildcard can be used twice a week. The three chips can only be used in a single gameweek. Here are some more in depth descriptions of the rules:

1. Captaincy: One player every week can be named captain. Their points are doubled for that gameweek.

2. Bench Boost Chip: The points scored by the bench players in the next gameweek are included in the points total.

3. Free Hit Chip: Make unlimited free transfers for a single gameweek. At the next deadline, the squad is returned to how it was at the start of the gameweek.

4. Triple Captain Chip: The captain points are tripled instead of doubled for that gameweek.

5. Wildcard: All transfers in the gameweek are free of charge. Each manager gets two wildcards; one in the first half of the season and one in the second half of the season.

For this project, we will be using three classes. The team of the week (totw) class helps us find the players that score the highest number of points in a given gameweek. The team of the year (toty) class helps us find the players that cumulatively score the highest number of points in a range of gameweeks. The evaulate class helps us evaluate the number of points achieved in a season, given a list of players for every gameweek. Here is a list of the classes:

In [2]:
from totw import totw
from toty import toty
from evaluate import evaluate

Let us take a closer look at the team of the week function, by looking at the first team of the week:

In [12]:
team_of_the_week_one = totw(1)
print("Week One Team of the Week:")
print(team_of_the_week_one.extract_names())

Week One Team of the Week:
['Ben_Foster', 'James_Collins', 'Stephen_Kingsley', 'Gareth_McAuley', 'Philippe_Coutinho', 'Anthony_Martial', 'Leroy_Fer', 'Adam_Lallana', 'Eden_Hazard', 'Álvaro_Negredo', 'Sergio_Agüero']


From the result of the previous cell, we have found the names of the eleven highest scoring players in the first gameweek of the season. These players were able to achieve high scores by either scoring goals, giving assists, keeping clean sheets, or making saves. We can get a more accurate look of how well they did by extracting the number of points they each achieved:

In [32]:
for x, y, in zip(team_of_the_week_one.extract_names(), team_of_the_week_one.extract_points()):
    print(x, ":", y)
print("")
print("The First Team of the Week achieved", sum(team_of_the_week_one.extract_points()), "points in total.")

Ben_Foster : 10
James_Collins : 8
Stephen_Kingsley : 8
Gareth_McAuley : 7
Philippe_Coutinho : 15
Anthony_Martial : 11
Leroy_Fer : 11
Adam_Lallana : 11
Eden_Hazard : 10
Álvaro_Negredo : 9
Sergio_Agüero : 9

The First Team of the Week achieved 109 points in total.


We can see from above that Philippe Coutinho was the highest scoring player of the first gameweek, with a whopping 15 points! There were a lot of points scored in this gameweek. How many points would we achieve if we find the sum of all the points achieved in each gameweek? Let's find out.

In [24]:
total_points = 0
for k in range(1,39):
    total_points += sum(totw(k).extract_points())
print("Cumulative number of points from the 38 gameweeks:", total_points)

Cumulative number of points from the 38 gameweeks: 5127


So that's it. If we play optimally, we get 5127 points!

Not so fast... This is not the optimal team, because it does not take into account many of the constraints. The position constraint is not satisfied, because we only picked the starting XI, and not the substitutes. The price constraint is not satisfied, because we did not look at prices when we picked the team of the week. The transfer constraint is not satisfied, because we did not take any points away for having an ever-changing team. The team constraint is not satisfied, because we are allowed to have more than three players from any team. In fact, the only constraint that we have satisfied is the formation constraint! Not to mention, we still have to add each of the bonus rules to our points total.

The following method finds the total that satisfies all of the constraints (except the transfer constraint) by adding **constraints=True** in the argument. Let's see how the points total is affected.

In [26]:
total_points = 0
for k in range(1,39):
    total_points += sum(totw(k,constraints=True).extract_points())
print("Cumulative number of points from the 38 gameweeks with constraints:", total_points)

Cumulative number of points from the 38 gameweeks with constraints: 5116


Not too bad! 5116 points from the 38 gameweeks seems to be a pretty good initial score. However, there are a lot of other aspects to consider before making assumptions.

We took a look at finding each of the teams of the week and adding the values together. Let's instead take a look at the team of the year, which is a team including the highest scoring players over the course of a season.

In [31]:
team_of_the_year = toty(1,38)
team_of_the_year.find_toty()
print("Team of the Year:")
print(team_of_the_year.find_names())

Team of the Year:
['Tom_Heaton', 'Gary_Cahill', 'Marcos_Alonso', 'César_Azpilicueta', 'Alexis_Sánchez', 'Bamidele_Alli', 'Eden_Hazard', 'Christian_Eriksen', 'Kevin_De Bruyne', 'Harry_Kane', 'Romelu_Lukaku']


From the result of the previous cell, we have found the names of the eleven highest scoring players in the entire season. These players were able to achieve high scores by either scoring goals, giving assists, keeping clean sheets, and/or making saves. We can get a more accurate look of how well they did by extracting the number of points they each achieved:

In [34]:
for x, y, in zip(team_of_the_year.find_names(), team_of_the_year.find_points()):
    print(x, ":", y)
print("")
print("The Team of the Year achieved", sum(team_of_the_year.find_points()), "points in total.")

Tom_Heaton : 149
Gary_Cahill : 178
Marcos_Alonso : 177
César_Azpilicueta : 170
Alexis_Sánchez : 264
Bamidele_Alli : 225
Eden_Hazard : 224
Christian_Eriksen : 218
Kevin_De Bruyne : 199
Harry_Kane : 224
Romelu_Lukaku : 221

The Team of the Year achieved 2249 points in total.


Wow! Those are some very high points totals for these players. However, we still have not satisfied most of the constraints that we should abide by. This will likely decrease our points total. By taking into account all of the team-buidling constraints, we can find a more optimal team of the year. This is done by adding **constraints=True** in the arguments. Let's take a look:

In [35]:
toty_with_constraints = toty(1,38,constraints=True)
toty_with_constraints.find_toty()
for x, y, in zip(toty_with_constraints.find_names(), toty_with_constraints.find_points()):
    print(x, ":", y)
print("")
print("The Team of the Year achieved", sum(toty_with_constraints.find_points()), "points in total.")

Tom_Heaton : 149
Gary_Cahill : 178
Leighton_Baines : 135
Charlie_Daniels : 134
Alexis_Sánchez : 264
Adam_Lallana : 139
Darren_Fletcher : 103
Christian_Eriksen : 218
Emre_Can : 104
Harry_Kane : 224
Romelu_Lukaku : 221

The Team of the Year achieved 1869 points in total.


That is quite a large decrease in the number of points when we took into account the constraints. But how would we fare if we take into account the transfer constraint and the extra rules. Let's see:

In [43]:
toty_starting, toty_subs = [], []
for k in range(0,38):
    toty_starting.append(toty_with_constraints.find_elements())
    toty_subs.append(toty_with_constraints.extract_subs_elements())
toty_evaluation = evaluate(toty_starting, toty_subs)
print("\nThe Team of the Year will achieve", toty_evaluation.total_points(), "points.")

The Triple Captain Chip will be played in Gameweek 37 where the player Harry_Kane scored 31 points
The First Wilcard Chip will be played in Gameweek 2
The Second Wildcard Chip will be played in Gameweek 20
The Bench Boost Chip will be played in Gameweek 1 earning an extra 0 points
The Free Hit Chip will be played in Gameweek 36 earning an extra 116 points

The Team of the Year will achieve 2562 points.


That's a lot of points! 2562 will be the most points that we have achieved so far, but let's take a look at how the team of the week will do in the team of the year.

In [44]:
totw_starting, totw_subs = [], []
for k in range(1,39):
    totw_starting.append(totw(k,True).extract_indices())
    totw_subs.append(totw(k,True).extract_subs())
totw_evaluation = evaluate(totw_starting, totw_subs)
print("\nThe Team of the Week will achieve", totw_evaluation.total_points(), "points.")

The Triple Captain Chip will be played in Gameweek 37 where the player Harry_Kane scored 31 points
The First Wilcard Chip will be played in Gameweek 3
The Second Wildcard Chip will be played in Gameweek 21
The Bench Boost Chip will be played in Gameweek 11 earning an extra 6 points
The Free Hit Chip will be played in Gameweek 1 earning an extra 0 points

The Team of the Week will achieve 4504 points.


Wow! Even more points. This will be our baseline. Let's see if we can find a better value...

In [45]:
list3 = []
list4 = []
for k in range(1,20):
    r = toty(2*k-1,2*k,True)
    r.find_toty()
    list3.append(r.find_elements())
    list3.append(r.find_elements())
    list4.append(r.extract_subs_elements())
    list4.append(r.extract_subs_elements())
n = evaluate(list3, list4)
print(n.total_points())

The Triple Captain Chip will be played in Gameweek 37 where the player Harry_Kane scored 31 points
The First Wilcard Chip will be played in Gameweek 3
The Second Wildcard Chip will be played in Gameweek 21
The Bench Boost Chip will be played in Gameweek 37 earning an extra 3 points
The Free Hit Chip will be played in Gameweek 28 earning an extra 62 points
4104
